# Upload a local datafile to add or replace a Dataset in a Collection

_\*\*The sample code in this notebook limits s3 upload durations to 12 hours. If you think your large file upload may take longer than that, please make use of the `upload_local_datafile` function, whose underlying code supports unlimited upload duration, as seen in the `python/create_dataset_from_local_file.ipynb` notebook.\*\*_

The script in this notebook performs the upload of a local datafile to a given Collection (as identified by its Collection id), where the datafile becomes a Dataset accessible via the CZ CELLxGENE Discover data portal.

In order to use this script, you must have a Curation API key (obtained from upper-righthand dropdown in the CZ CELLxGENE Discover data portal after logging in).

_For **new** Datasets_: You must separately create a Dataset (not shown in this notebook). Then, use the returned `dataset_id` as the suffix (append to the `UploadKeyPrefix` returned from the `/s3-upload-credentials` endpoint) of the S3 upload key. See code below, or read more detailed instructions about how to submit Datasets via S3 upload in [the description for the credentials endpoint](https://api.cellxgene.cziscience.com/curation/ui/#/collection/backend.corpora.lambdas.api.v1.curation.collections.collection_id.datasets.upload_s3.get).

_For **replacing/updating** existing Datasets_: Uploads to a `dataset_id` for which there already exists a Dataset in the given Collection will result in the existing Dataset being replaced by a new Dataset created from the datafile that you are uploading.


You can only add/replace Datasets in _private_ Collections or _private revisions_ of published Collections.

### Import dependencies

In [ ]:
library("readr")
library("aws.s3")
library("httr")
library("stringr")

#### <font color='#bc00b0'>Please fill in the required values:</font>

<font color='#bc00b0'>(Required) Provide the path to your api key file</font>

In [ ]:
api_key_file_path <- "path/to/api-key.txt"

<font color='#bc00b0'>(Required) Provide the absolute path to the h5ad datafile to upload</font>

In [1]:
filename <- "/absolute/path/to-datafile.h5ad"

<font color='#bc00b0'>(Required) Enter the id of the Collection to which you wish to add this datafile as a Dataset</font>

_The Collection id can be found by looking at the url path in the address bar 
when viewing your Collection in the CZ CELLxGENE Discover data portal: `/collections/{collection_id}`. You can only add/replace Datasets in private Collections or private revisions of published Collections. In order to edit a published Collection, you must first create a revision of that Collection._

In [ ]:
collection_id <- "01234567-89ab-cdef-0123-456789abcdef"

<font color='#bc00b0'>(Required) Enter the id of the Dataset to which you wish to upload your datafile</font>

_The Dataset id can be found by using the `GET /collections/{collection_id}` endpoint and filtering for the Dataset of interest OR by looking at the url path in the address when viewing your Dataset using the CZ CELLxGENE Explorer browser tool: `/e/{dataset_id}.cxg/`. See heading at top for rules about adding vs updating Datasets._

In [2]:
dataset_id <- "abcdef01-2345-6789-abcd-ef0123456789"

### Specify domain (and API url)

In [ ]:
domain_name <- "cellxgene.cziscience.com"
site_url <- str_interp("https://${domain_name}")
api_url_base <- str_interp("https://api.${domain_name}")

### Use API key to obtain a temporary access token

In [ ]:
api_key <- read_file(api_key_file_path)
access_token_path <- "/curation/v1/auth/token"
access_token_url <- str_interp("${api_url_base}${access_token_path}")
res <- POST(url=access_token_url, add_headers(`x-api-key`=api_key))
stop_for_status(res)
access_token <- content(res)$access_token

##### (optional, debug) verify status code of response

In [ ]:
print(res$status_code)

### Retrieve temporary s3 write credentials

In [ ]:
s3_credentials_path <- str_interp("/curation/v1/collections/${collection_id}/datasets/s3-upload-credentials")
url <- str_interp("${api_url_base}${s3_credentials_path}")
bearer_token <- str_interp("Bearer ${access_token}")
res <- GET(url=url, add_headers(`Authorization`=bearer_token, `Content-Type`="application/json"))
stop_for_status(res)
res_content <- content(res)
access_key_id <- res_content$Credentials$AccessKeyId
secret_access_key <- res_content$Credentials$SecretAccessKey
session_token <- res_content$Credentials$SessionToken
upload_path <- res_content$UploadPath

### Extract formatted upload path from credentials endpoint response

In [ ]:
bucket <- res_content$Bucket
key_prefix <- res_content$UploadKeyPrefix
upload_key <- paste(key_prefix, dataset_id, sep="")
print(str_interp("Full S3 write path is s3://${bucket}/${upload_key}"))

### Upload file using temporary AWS S3 credentials

In [ ]:
Sys.setenv(
    "AWS_ACCESS_KEY_ID" = access_key_id,
    "AWS_SECRET_ACCESS_KEY" = secret_access_key,
    "AWS_SESSION_TOKEN" = session_token,
    "AWS_DEFAULT_REGION" = "us-west-2"
)
put_object(file=filename, object=upload_key, bucket=bucket)